In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

# Tải mô hình đã huấn luyện
model = load_model("emotion_recognition_model.keras")
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Bộ phát hiện khuôn mặt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Đọc ảnh (có thể là màu hoặc grayscale nhỏ 48x48)
img_path = "Disgust.jpg"  # <-- Thay bằng "Surprise.jpg" để test ảnh khác
img = cv2.imread(img_path)

if img is None:
    raise ValueError(f"Không thể đọc ảnh: {img_path}")

# Nếu ảnh có kích thước nhỏ (ví dụ 48x48), xử lý trực tiếp
if img.shape[0] <= 64 or img.shape[1] <= 64:
    # Chuyển sang grayscale nếu chưa có 1 kênh
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_resized = cv2.resize(img, (48, 48))
    input_img = img_resized.reshape(1, 48, 48, 1) / 255.0

    prediction = model.predict(input_img)
    emotion = emotion_labels[np.argmax(prediction)]
    print(f"Emotion (no detection): {emotion}")

else:
    # Xử lý bình thường với ảnh lớn
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    if len(faces) == 0:
        print("Không phát hiện được khuôn mặt nào.")
    else:
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (48, 48))
            input_img = face_resized.reshape(1, 48, 48, 1) / 255.0

            prediction = model.predict(input_img)
            emotion = emotion_labels[np.argmax(prediction)]

            print(f"Emotion: {emotion}")
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Hiển thị kết quả (nếu là ảnh lớn)
        cv2.imshow("Detected", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


1/1 [==============================] - 1s 722ms/step
Emotion (no detection): Disgust
